### This notebook is just me re-running every validation test with improved modelling and saving the reulsts to Complete.csv

In [1]:
import numpy as np
import geopandas as gp
import pandas as pd
import datetime as dt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit

# import .py scripts from repo 
from json_to_csv import geojson_to_csv
from ts_train_test_split import uni_selection
from ts_train_test_split import single_split
from model_validation import model_val
from model_testing_function import test_all
from model_testing_function import append_results

In [2]:
old = pd.read_csv('Locations.csv', index_col=0)

In [3]:
old

,lat,lon,Place,p,d,q,P,D,Q,filepath
0,49.0362,-122.3247,"Abbotsford, Canada",3,1,3,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
1,49.0362,-118.4245,"Grand Forks, Canada",7,0,8,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
2,48.9289,-117.8306,"Northport, WA",6,1,7,2,0,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
3,48.7835,-122.3357,"Dewey, WA",8,0,6,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
4,48.7826,-121.7994,"Mt. Baker, WA",8,1,8,0,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
5,48.5767,-122.9509,"Shaw Island, WA",7,0,7,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
6,48.5085,-117.3252,"Rudy, WA",8,1,5,1,1,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
7,48.4093,-123.5716,"Metchosin, Canada",7,1,4,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
8,48.0139,-123.4453,"Port Angeles, WA",7,1,8,0,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
9,47.9735,-122.2038,"Everett, WA",4,0,5,0,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...


In [11]:
locations = pd.read_csv('Locations_2.csv', index_col=0)

In [12]:
locations

,Unnamed: 0.1,lat,lon,Place,p,d,q,P,D,Q,filepath
0,0,49.0362,-122.3247,"Abbotsford, Canada",1,0,8,2,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
1,1,49.0362,-118.4245,"Grand Forks, Canada",1,0,0,1,0,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
2,2,48.9289,-117.8306,"Northport, WA",1,0,2,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
3,3,48.7835,-122.3357,"Dewey, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
4,4,48.7826,-121.7994,"Mt. Baker, WA",4,0,6,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
5,5,48.5767,-122.9509,"Shaw Island, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
6,6,48.5085,-117.3252,"Rudy, WA",8,0,6,1,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
7,7,48.4093,-123.5716,"Metchosin, Canada",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
8,8,48.0139,-123.4453,"Port Angeles, WA",2,0,1,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...
9,9,47.9735,-122.2038,"Everett, WA",7,0,8,1,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...


In [13]:
results = test_all(locations)

/home/cbenstead/miniconda3/envs/SERF/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/cbenstead/miniconda3/envs/SERF/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/home/cbenstead/miniconda3/envs/SERF/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/cbenstead/miniconda3/envs/SERF/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting paramete

In [14]:
complete = append_results(locations, results)

In [23]:
complete


,Unnamed: 0.1,lat,lon,Place,p,d,q,P,D,Q,filepath,MSE,r2
0,0,49.0362,-122.3247,"Abbotsford, Canada",1,0,8,2,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003737,0.563818
1,1,49.0362,-118.4245,"Grand Forks, Canada",1,0,0,1,0,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003181,0.631988
2,2,48.9289,-117.8306,"Northport, WA",1,0,2,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003246,0.732744
3,3,48.7835,-122.3357,"Dewey, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003064,0.662395
4,4,48.7826,-121.7994,"Mt. Baker, WA",4,0,6,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003416,0.611871
5,5,48.5767,-122.9509,"Shaw Island, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003064,0.662395
6,6,48.5085,-117.3252,"Rudy, WA",8,0,6,1,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003330,0.725801
7,7,48.4093,-123.5716,"Metchosin, Canada",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003064,0.662395
8,8,48.0139,-123.4453,"Port Angeles, WA",2,0,1,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003071,0.661566
9,9,47.9735,-122.2038,"Everett, WA",7,0,8,1,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003160,0.607775


In [24]:
complete = complete.drop(labels='Unnamed: 0.1', axis=1)
complete

,lat,lon,Place,p,d,q,P,D,Q,filepath,MSE,r2
0,49.0362,-122.3247,"Abbotsford, Canada",1,0,8,2,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003737,0.563818
1,49.0362,-118.4245,"Grand Forks, Canada",1,0,0,1,0,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003181,0.631988
2,48.9289,-117.8306,"Northport, WA",1,0,2,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003246,0.732744
3,48.7835,-122.3357,"Dewey, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003064,0.662395
4,48.7826,-121.7994,"Mt. Baker, WA",4,0,6,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003416,0.611871
5,48.5767,-122.9509,"Shaw Island, WA",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003064,0.662395
6,48.5085,-117.3252,"Rudy, WA",8,0,6,1,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003330,0.725801
7,48.4093,-123.5716,"Metchosin, Canada",7,0,5,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003064,0.662395
8,48.0139,-123.4453,"Port Angeles, WA",2,0,1,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003071,0.661566
9,47.9735,-122.2038,"Everett, WA",7,0,8,1,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003160,0.607775


In [25]:
complete.to_csv('Final_validation.csv')

In [10]:
previous = pd.read_csv('Complete.csv', index_col=0)
previous

,lat,lon,Place,p,d,q,P,D,Q,filepath,MSE,r2
0,49.0362,-122.3247,"Abbotsford, Canada",3,1,3,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.005327,0.378109
1,49.0362,-118.4245,"Grand Forks, Canada",7,0,8,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003268,0.621841
2,48.9289,-117.8306,"Northport, WA",6,1,7,2,0,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.004475,0.631540
3,48.7835,-122.3357,"Dewey, WA",8,0,6,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003995,0.559698
4,48.7826,-121.7994,"Mt. Baker, WA",8,1,8,0,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.004979,0.434240
5,48.5767,-122.9509,"Shaw Island, WA",7,0,7,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.004066,0.551867
6,48.5085,-117.3252,"Rudy, WA",8,1,5,1,1,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003119,0.743171
7,48.4093,-123.5716,"Metchosin, Canada",7,1,4,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003985,0.560855
8,48.0139,-123.4453,"Port Angeles, WA",7,1,8,0,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003237,0.643225
9,47.9735,-122.2038,"Everett, WA",4,0,5,0,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.005954,0.261099


In [17]:
loc = pd.read_csv('Locations.csv', index_col=0)

In [21]:
complete = append_results(loc, results)
complete

,lat,lon,Place,p,d,q,P,D,Q,filepath,MSE,r2
0,49.0362,-122.3247,"Abbotsford, Canada",3,1,3,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.005327,0.378109
1,49.0362,-118.4245,"Grand Forks, Canada",7,0,8,0,1,1,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003268,0.621841
2,48.9289,-117.8306,"Northport, WA",6,1,7,2,0,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.004475,0.631540
3,48.7835,-122.3357,"Dewey, WA",8,0,6,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003995,0.559698
4,48.7826,-121.7994,"Mt. Baker, WA",8,1,8,0,0,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.004979,0.434240
5,48.5767,-122.9509,"Shaw Island, WA",7,0,7,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.004066,0.551867
6,48.5085,-117.3252,"Rudy, WA",8,1,5,1,1,2,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003119,0.743171
7,48.4093,-123.5716,"Metchosin, Canada",7,1,4,1,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003985,0.560855
8,48.0139,-123.4453,"Port Angeles, WA",7,1,8,0,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.003237,0.643225
9,47.9735,-122.2038,"Everett, WA",4,0,5,0,1,0,NASA/POWER_Point_Monthly_Timeseries_1981_2020_...,0.005954,0.261099


In [20]:
complete.to_csv('Complete.csv')